### N-gram language models or how to write scientific papers (4 pts)

We shall train our language model on a corpora of [ArXiv](http://arxiv.org/) articles and see if we can generate a new one!

![img](https://media.npr.org/assets/img/2013/12/10/istock-18586699-monkey-computer_brick-16e5064d3378a14e0e4c2da08857efe03c04695e-s800-c85.jpg)

_data by neelshah18 from [here](https://www.kaggle.com/neelshah18/arxivdataset/)_

_Disclaimer: this has nothing to do with actual science. But it's fun, so who cares?!_

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Alternative manual download link: https://yadi.sk/d/_nGyU2IajjR9-w
!wget "https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1" -O arxivData.json.tar.gz
!tar -xvzf arxivData.json.tar.gz
data = pd.read_json("./arxivData.json")
data.sample(n=5)

--2021-01-06 10:20:12--  https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.64.18, 2620:100:6022:18::a27d:4212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.64.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/99az9n1b57qkd9j/arxivData.json.tar.gz [following]
--2021-01-06 10:20:13--  https://www.dropbox.com/s/dl/99az9n1b57qkd9j/arxivData.json.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc2e59157f596fa9ba4c197ea2c2.dl.dropboxusercontent.com/cd/0/get/BGcMxexmaQyZxYWC6pris6Nsku0GEUuHfWOFNDSDMfKP6-ilK9oSm61lPIS6eW_o-BQrhLyvwEcmRfkM9JszCz5VqDt7Bv_NkYHsvba7I6wJng/file?dl=1# [following]
--2021-01-06 10:20:13--  https://uc2e59157f596fa9ba4c197ea2c2.dl.dropboxusercontent.com/cd/0/get/BGcMxexmaQyZxYWC6pris6Nsku0GEUuHfWOFNDSDMfKP6-ilK9oSm61lPIS6eW_o-BQrhLyvwEcmRfkM9JszCz5VqDt7Bv_NkYHs

,author,day,id,link,month,summary,tag,title,year
18320,"[{'name': 'Martin Josef Geiger'}, {'name': 'Wo...",3,0809.0610v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",9,The article presents a framework for the resol...,"[{'term': 'cs.AI', 'scheme': 'http://arxiv.org...",A framework for the interactive resolution of ...,2008
18378,[{'name': 'Florentin Smarandache'}],31,1002.0102v5,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",1,In this book we introduce a new procedure call...,"[{'term': 'cs.AI', 'scheme': 'http://arxiv.org...",$α$-Discounting Multi-Criteria Decision Making...,2010
1340,"[{'name': 'Jimmy Ba'}, {'name': 'Volodymyr Mni...",24,1412.7755v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",12,We present an attention-based model for recogn...,"[{'term': 'cs.LG', 'scheme': 'http://arxiv.org...",Multiple Object Recognition with Visual Attention,2014
17595,"[{'name': 'Maryam Zare'}, {'name': 'Shaurya Ro...",17,1712.06994v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",12,This paper presents an simple yet sophisticate...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",DeepNorm-A Deep Learning Approach to Text Norm...,2017
32614,[{'name': 'Jim Jing-Yan Wang'}],3,1409.1200v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",9,"In this paper, we propose the problem of domai...","[{'term': 'cs.LG', 'scheme': 'http://arxiv.org...",Domain Transfer Structured Output Learning,2014


In [3]:
# assemble lines: concatenate title and description
lines = data.apply(lambda row: row['title'] + ' ; ' + row['summary'], axis=1).tolist()

sorted(lines, key=len)[:3]

['Differential Contrastive Divergence ; This paper has been retracted.',
 'What Does Artificial Life Tell Us About Death? ; Short philosophical essay',
 'P=NP ; We claim to resolve the P=?NP problem via a formal argument for P=NP.']

### Tokenization

You know the dril. The data is messy. Go clean the data. Use WordPunctTokenizer or something.


In [4]:
# Task: convert lines (in-place) into strings of space-separated tokens. import & use WordPunctTokenizer
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()
lines = [' '.join(tokenizer.tokenize(l.lower())) for l in lines]

In [5]:
assert sorted(lines, key=len)[0] == \
    'differential contrastive divergence ; this paper has been retracted .'
assert sorted(lines, key=len)[2] == \
    'p = np ; we claim to resolve the p =? np problem via a formal argument for p = np .'

### N-Gram Language Model (1point)

A language model is a probabilistic model that estimates text probability: the joint probability of all tokens $w_t$ in text $X$: $P(X) = P(w_1, \dots, w_T)$.

It can do so by following the chain rule:
$$P(w_1, \dots, w_T) = P(w_1)P(w_2 \mid w_1)\dots P(w_T \mid w_1, \dots, w_{T-1}).$$ 

The problem with such approach is that the final term $P(w_T \mid w_1, \dots, w_{T-1})$ depends on $n-1$ previous words. This probability is impractical to estimate for long texts, e.g. $T = 1000$.

One popular approximation is to assume that next word only depends on a finite amount of previous words:

$$P(w_t \mid w_1, \dots, w_{t - 1}) = P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1})$$

Such model is called __n-gram language model__ where n is a parameter. For example, in 3-gram language model, each word only depends on 2 previous words. 

$$
    P(w_1, \dots, w_n) = \prod_t P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1}).
$$

You can also sometimes see such approximation under the name of _n-th order markov assumption_.

The first stage to building such a model is counting all word occurences given N-1 previous words

In [6]:
from tqdm import tqdm
from collections import defaultdict, Counter

# special tokens: 
# - unk represents absent tokens, 
# - eos is a special token after the end of sequence

UNK, EOS = "_UNK_", "_EOS_"


# Wrote this helper function
def get_prefix(line, n):
    sentence = line.split()
    sentence.append(EOS)
    for i, word in enumerate(sentence):
        prefix = []
        if i < n - 1:
            for k in range(n - i - 1):
                prefix.append(UNK)
            if k < n - 1:
                for l in range(n - k - 2):
                    prefix.append(sentence[l])
            yield(tuple(prefix), word)
            continue
        for j in range(i+1-n, i):
            prefix.append(sentence[j])

        yield(tuple(prefix), word)

def count_ngrams(lines, n):
    """
    Count how many times each word occured after (n - 1) previous words
    :param lines: an iterable of strings with space-separated tokens
    :returns: a dictionary { tuple(prefix_tokens): {next_token_1: count_1, next_token_2: count_2}}

    When building counts, please consider the following two edge cases
    - if prefix is shorter than (n - 1) tokens, it should be padded with UNK. For n=3,
      empty prefix: "" -> (UNK, UNK)
      short prefix: "the" -> (UNK, the)
      long prefix: "the new approach" -> (new, approach)
    - you should add a special token, EOS, at the end of each sequence
      "... with deep neural networks ." -> (..., with, deep, neural, networks, ., EOS)
      count the probability of this token just like all others.
    """
    counts = defaultdict(Counter)
    # counts[(word1, word2)][word3] = how many times word3 occured after (word1, word2)
    

    for line in lines:
        for prefix, word in get_prefix(line, n):
            counts[prefix][word] = \
            counts[prefix].get(word, 0) + 1

    return counts


In [7]:
# let's test it
dummy_lines = sorted(lines, key=len)[:100]
dummy_counts = count_ngrams(dummy_lines, n=3)
assert set(map(len, dummy_counts.keys())) == {2}, "please only count {n-1}-grams"
assert len(dummy_counts[('_UNK_', '_UNK_')]) == 78
assert dummy_counts['_UNK_', 'a']['note'] == 3
assert dummy_counts['p', '=']['np'] == 2
assert dummy_counts['author', '.']['_EOS_'] == 1

In [8]:
probs = defaultdict(Counter)
for prefix, counts in dummy_counts.items():
    S = sum(counts.values())
    for word in counts.keys():
        probs[prefix][word] = dummy_counts[prefix][word]/S


Once we can count N-grams, we can build a probabilistic language model.
The simplest way to compute probabilities is in proporiton to counts:

$$ P(w_t | prefix) = { Count(prefix, w_t) \over \sum_{\hat w} Count(prefix, \hat w) } $$

In [9]:
class NGramLanguageModel:    
    def __init__(self, lines, n):
        """ 
        Train a simple count-based language model: 
        compute probabilities P(w_t | prefix) given ngram counts
        
        :param n: computes probability of next token given (n - 1) previous words
        :param lines: an iterable of strings with space-separated tokens
        """
        assert n >= 1
        self.n = n
    
        counts = count_ngrams(lines, self.n)
        
        # compute token proabilities given counts
        self.probs = defaultdict(Counter)
        # probs[(word1, word2)][word3] = P(word3 | word1, word2)
        
        # populate self.probs with actual probabilities
        for prefix, counter in counts.items():
            S = sum(counter.values())
            for word, frequency in counter.items():
                self.probs[prefix][word] = frequency/S
            
    def get_possible_next_tokens(self, prefix):
        """
        :param prefix: string with space-separated prefix tokens
        :returns: a dictionary {token : it's probability} for all tokens with positive probabilities
        """
        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [ UNK ] * (self.n - 1 - len(prefix)) + prefix
        return self.probs[tuple(prefix)]
    
    def get_next_token_prob(self, prefix, next_token):
        """
        :param prefix: string with space-separated prefix tokens
        :param next_token: the next token to predict probability for
        :returns: P(next_token|prefix) a single number, 0 <= P <= 1
        """
        return self.get_possible_next_tokens(prefix).get(next_token, 0)

Let's test it!

In [10]:
dummy_lm = NGramLanguageModel(dummy_lines, n=3)

p_initial = dummy_lm.get_possible_next_tokens('') # '' -> ['_UNK_', '_UNK_']
assert np.allclose(p_initial['learning'], 0.02)
assert np.allclose(p_initial['a'], 0.13)
assert np.allclose(p_initial.get('meow', 0), 0)
assert np.allclose(sum(p_initial.values()), 1)

p_a = dummy_lm.get_possible_next_tokens('a') # '' -> ['_UNK_', 'a']
assert np.allclose(p_a['machine'], 0.15384615)
assert np.allclose(p_a['note'], 0.23076923)
assert np.allclose(p_a.get('the', 0), 0)
assert np.allclose(sum(p_a.values()), 1)

assert np.allclose(dummy_lm.get_possible_next_tokens('a note')['on'], 1)
assert dummy_lm.get_possible_next_tokens('a machine') == \
    dummy_lm.get_possible_next_tokens("there have always been ghosts in a machine"), \
    "your 3-gram model should only depend on 2 previous words"

Now that you've got a working n-gram language model, let's see what sequences it can generate. But first, let's train it on the whole dataset.

In [11]:
lm = NGramLanguageModel(lines, n=3)

The process of generating sequences is... well, it's sequential. You maintain a list of tokens and iteratively add next token by sampling with probabilities.

$ X = [] $

__forever:__
* $w_{next} \sim P(w_{next} | X)$
* $X = concat(X, w_{next})$


Instead of sampling with probabilities, one can also try always taking most likely token, sampling among top-K most likely tokens or sampling with temperature. In the latter case (temperature), one samples from

$$w_{next} \sim {P(w_{next} | X) ^ {1 / \tau} \over \sum_{\hat w} P(\hat w | X) ^ {1 / \tau}}$$

Where $\tau > 0$ is model temperature. If $\tau << 1$, more likely tokens will be sampled with even higher probability while less likely tokens will vanish.

In [12]:
def get_next_token(lm, prefix, temperature=1.0):
    """
    return next token after prefix;
    :param temperature: samples proportionally to lm probabilities ^ (1 / temperature)
        if temperature == 0, always takes most likely token. Break ties arbitrarily.
    """
    words = []
    probs = []
    if temperature == 0:
        return lm.get_possible_next_tokens(prefix).most_common(1)[0][0]
    for word, proba in lm.get_possible_next_tokens(prefix).items():
        words.append(word)
        probs.append(proba**(1 / temperature))
    # tokens, probs = zip(*lm.get_possible_next_tokens(prefix))
    return np.random.choice(words, size=1, replace=True, p=probs/np.sum(probs))[0]

In [13]:
from collections import Counter
test_freqs = Counter([get_next_token(lm, 'there have') for _ in range(10000)])
assert 250 < test_freqs['not'] < 450
assert 8500 < test_freqs['been'] < 9500
assert 1 < test_freqs['lately'] < 200

test_freqs = Counter([get_next_token(lm, 'deep', temperature=1.0) for _ in range(10000)])
assert 1500 < test_freqs['learning'] < 3000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.5) for _ in range(10000)])
assert 8000 < test_freqs['learning'] < 9000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.0) for _ in range(10000)])
assert test_freqs['learning'] == 10000

print("Looks nice!")

Looks nice!


Let's have fun with this model

In [14]:
prefix = 'artificial' # <- your ideas :)

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

artificial neural network . our contribution includes new features are naturally modeled by subjective probability curves . this class has been adopted as a debugging approach currently used in the occurrence tensor from a common issue in recent years . each node ' s pride and prejudice . there is an important social computing task - relevant input attributes appropriately . to address this issue we propose a multipoint bandit feedback algorithm with a large class of subadditive valuations . we apply dssl to the community . ctrax was not explicitly list word pairs ( e . g . the proposed


In [15]:
prefix = 'bridging the' # <- more of your ideas

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix, temperature=0.5)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

bridging the gap between these two methods for the task of learning a mapping of high - dimensional latent variables in a given threshold . we show that our method is developed to reconstruct the input data . the main idea is to estimate the relative approximation error . in this paper , we propose a novel approach to the state - of - the - art methods on both synthetic and real data sets . _EOS_


__More in the homework:__ nucleous sampling, top-k sampling, beam search(not for the faint of heart).

### Evaluating language models: perplexity (1point)

Perplexity is a measure of how well does your model approximate true probability distribution behind data. __Smaller perplexity = better model__.

To compute perplexity on one sentence, use:
$$
    {\mathbb{P}}(w_1 \dots w_N) = P(w_1, \dots, w_N)^{-\frac1N} = \left( \prod_t P(w_t \mid w_{t - n}, \dots, w_{t - 1})\right)^{-\frac1N},
$$


On the corpora level, perplexity is a product of probabilities of all tokens in all sentences to the power of 1, divided by __total length of all sentences__ in corpora.

This number can quickly get too small for float32/float64 precision, so we recommend you to first compute log-perplexity (from log-probabilities) and then take the exponent.

In [37]:
k = lm.n
N = 0
logperplexity = 0
min_logprob=np.log(10 ** -50.)
N += len(lines[0].split())
for prefix, word in get_prefix(lines[0],k):
    logprob = np.log2(lm.get_next_token_prob(' '.join(prefix), word))
    if logprob < min_logprob:
        logprob = min_logprob
    logperplexity += logprob

In [123]:
def perplexity(lm, lines, min_logprob=np.log(10 ** -50.)):
    """
    :param lines: a list of strings with space-separated tokens
    :param min_logprob: if log(P(w | ...)) is smaller than min_logprop, set it equal to min_logrob
    :returns: corpora-level perplexity - a single scalar number from the formula above
    
    Note: do not forget to compute P(w_first | empty) and P(eos | full_sequence)
    
    PLEASE USE lm.get_next_token_prob and NOT lm.get_possible_next_tokens
    """
    n = lm.n
    length_sentences = 0
    logperplexity = 0
    
    for i, line in enumerate(lines):
        for prefix, word in get_prefix(line, n):
            length_sentences += 1
            logprob = np.log(lm.get_next_token_prob(' '.join(prefix), word))
            if logprob < min_logprob:
                logprob = min_logprob
            logperplexity += logprob
    return np.exp(-logperplexity/length_sentences)


In [124]:
lm1 = NGramLanguageModel(dummy_lines, n=1)
lm3 = NGramLanguageModel(dummy_lines, n=3)
lm10 = NGramLanguageModel(dummy_lines, n=10)

ppx1 = perplexity(lm1, dummy_lines)
ppx3 = perplexity(lm3, dummy_lines)
ppx10 = perplexity(lm10, dummy_lines)
ppx_missing = perplexity(lm3, ['the jabberwock , with eyes of flame , '])  # thanks, L. Carrol

print("Perplexities: ppx1=%.3f ppx3=%.3f ppx10=%.3f" % (ppx1, ppx3, ppx10))

assert all(0 < ppx < 500 for ppx in (ppx1, ppx3, ppx10)), "perplexity should be nonnegative and reasonably small"
assert ppx1 > ppx3 > ppx10, "higher N models should overfit and "
assert np.isfinite(ppx_missing) and ppx_missing > 10 ** 6, "missing words should have large but finite perplexity. " \
    " Make sure you use min_logprob right"
assert np.allclose([ppx1, ppx3, ppx10], (318.2132342216302, 1.5199996213739575, 1.1838145037901249))

Perplexities: ppx1=318.213 ppx3=1.520 ppx10=1.184


/home/userlocal/miniconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:18: RuntimeWarning: divide by zero encountered in log


Now let's measure the actual perplexity: we'll split the data into train and test and score model on test data only.

In [125]:
from sklearn.model_selection import train_test_split
train_lines, test_lines = train_test_split(lines, test_size=0.25, random_state=42)

for n in (1, 2, 3):
    lm = NGramLanguageModel(n=n, lines=train_lines)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))


/home/userlocal/miniconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:18: RuntimeWarning: divide by zero encountered in log


N = 1, Perplexity = 1832.23136
N = 2, Perplexity = 85653987.28774
N = 3, Perplexity = 61999196259043346743296.00000


In [ ]:
# whoops, it just blew up :)

### LM Smoothing

The problem with our simple language model is that whenever it encounters an n-gram it has never seen before, it assigns it with the probabilitiy of 0. Every time this happens, perplexity explodes.

To battle this issue, there's a technique called __smoothing__. The core idea is to modify counts in a way that prevents probabilities from getting too low. The simplest algorithm here is Additive smoothing (aka [Lapace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing)):

$$ P(w_t | prefix) = { Count(prefix, w_t) + \delta \over \sum_{\hat w} (Count(prefix, \hat w) + \delta) } $$

If counts for a given prefix are low, additive smoothing will adjust probabilities to a more uniform distribution. Not that the summation in the denominator goes over _all words in the vocabulary_.

Here's an example code we've implemented for you:

In [127]:
class LaplaceLanguageModel(NGramLanguageModel): 
    """ this code is an example, no need to change anything """
    def __init__(self, lines, n, delta=1.0):
        self.n = n
        counts = count_ngrams(lines, self.n)
        self.vocab = set(token for token_counts in counts.values() for token in token_counts)
        self.probs = defaultdict(Counter)

        for prefix in counts:
            token_counts = counts[prefix]
            total_count = sum(token_counts.values()) + delta * len(self.vocab)
            self.probs[prefix] = {token: (token_counts[token] + delta) / total_count
                                          for token in token_counts}
    def get_possible_next_tokens(self, prefix):
        token_probs = super().get_possible_next_tokens(prefix)
        missing_prob_total = 1.0 - sum(token_probs.values())
        missing_prob = missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        return {token: token_probs.get(token, missing_prob) for token in self.vocab}
    
    def get_next_token_prob(self, prefix, next_token):
        token_probs = super().get_possible_next_tokens(prefix)
        if next_token in token_probs:
            return token_probs[next_token]
        else:
            missing_prob_total = 1.0 - sum(token_probs.values())
            missing_prob_total = max(0, missing_prob_total) # prevent rounding errors
            return missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        

In [128]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = LaplaceLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

In [129]:
for n in (1, 2, 3):
    lm = LaplaceLanguageModel(train_lines, n=n, delta=0.1)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

N = 1, Perplexity = 977.67559
N = 2, Perplexity = 470.48021
N = 3, Perplexity = 3679.44765


In [ ]:
# optional: try to sample tokens from such a model

In [135]:
lm = LaplaceLanguageModel(train_lines, n=3, delta=0.1)

In [136]:
prefix = 'bridging the' # <- your ideas :)

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix, temperature=0.5)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

bridging the gap between the two tasks rfss fibers dmpnet nbs pcfg esqn want candecomp haptic decelerations fpk berth submatrics kyoto compactness metonymy gomal unbagged crosstable epitomes eduction leon demonstrations igmn encircle tclsta gart mizar allegorical dynamism supercedes reminders bleu neurodegenerative asks nonadjacent inventories gm ended sensitiveness reactants taleb fiqh excludes constriction chms msd agm rsi occfm breaches 2015parameterlessevolutionaryalgorithmsjava respectively vin cherry predicts aspires cgas wikip intricate obstruct mais activation chipmunk enormously foba russian activitynet1 departed deeplogo smdpalgorithms 86 circumsolar diction steam defense rollout nashville carreteras projected ferries gunning decomposition rtk git testsets firefighter merriman encounter idtm fannie =\{ mellon manner crawling


Interesting! Model with Laplace Smoothing is quite bad at generating texts.

### Kneser-Ney smoothing (2 points)

Additive smoothing is simple, reasonably good but definitely not a State of The Art algorithm.


Your final task in this notebook is to implement [Kneser-Ney](https://en.wikipedia.org/wiki/Kneser%E2%80%93Ney_smoothing) smoothing.

It can be computed recurrently, for n>1:

$$P_{kn}(w_t | prefix_{n-1}) = { \max(0, Count(prefix_{n-1}, w_t) - \delta) \over \sum_{\hat w} Count(prefix_{n-1}, \hat w)} + \lambda_{prefix_{n-1}} \cdot P_{kn}(w_t | prefix_{n-2})$$

where
- $prefix_{n-1}$ is a tuple of {n-1} previous tokens
- $lambda_{prefix_{n-1}}$ is a normalization constant chosen so that probabilities add up to 1
- Unigram $P_{kn}(w_t | prefix_{n-2})$ corresponds to Kneser Ney smoothing for {N-1}-gram language model.
- Unigram $P_{kn}(w_t)$ is a special case: how likely it is to see x_t in an unfamiliar context

See lecture slides or wiki for more detailed formulae.

__Your task__ is to
- implement KneserNeyLanguageModel
- test it on 1-3 gram language models
- find optimal (within reason) smoothing delta for 3-gram language model with Kneser-Ney smoothing

In [237]:
dummy_counts = count_ngrams(dummy_lines, n=1)
probs = defaultdict(Counter)
for prefix, counts in dummy_counts.items():
    S = sum(counts.values())
    for word in counts.keys():
        probs[prefix][word] = dummy_counts[prefix][word]/S
probs[()]['has']

0.005908932916232186

In [238]:
0.6086956521739131 + 0.005908932916232186

0.6146045850901453

In [255]:
dummy_counts = count_ngrams(dummy_lines, n=2)
probs = defaultdict(Counter)
for prefix, counts in dummy_counts.items():
    S = sum(counts.values())
    for word in counts.keys():
        probs[prefix][word] = dummy_counts[prefix][word]/S
probs[('paper', )]

Counter({'has': 0.6086956521739131,
         'specifies': 0.043478260869565216,
         'presents': 0.13043478260869565,
         ',': 0.043478260869565216,
         'we': 0.13043478260869565,
         'describes': 0.043478260869565216})

In [257]:
dummy_counts[('paper', )]

Counter({'has': 14,
         'specifies': 1,
         'presents': 3,
         ',': 1,
         'we': 3,
         'describes': 1})

In [272]:
recprobs = recursiveProba(dummy_lines, lmda=1, n=2, delta=0)
recprobs[('paper',)]

Counter({'has': 0.6146045850901453,
         'specifies': 0.043825845158755344,
         'presents': 0.13147753547626603,
         ',': 0.059119553883121005,
         'we': 0.14051472699520937,
         'describes': 0.04417342944794547})

In [271]:
def recursiveProba(lines, lmda, n, delta):
#     print("N Equals", n)
    if n == 1:
        counts = count_ngrams(lines, n)
        probs = defaultdict(Counter)
        for prefix, counter in counts.items():
            S = sum(counter.values())
            for word, frequency in counter.items():
                probs[prefix][word] = frequency/S
        return probs
#     print('DEALING WITH N', n)
    counts = count_ngrams(lines, n)
#     print(counts[('paper', )])
    probs = defaultdict(Counter)
    vocab = set(token for token_counts in counts.values() for token in token_counts)
    n = n - 1
#     lmda = 1/len(vocab)
    for prefix, counter in counts.items():
#         print(prefix)
        prefixRecursive = prefix[:n-1]
#         if prefix == ('paper', ):
#             print(prefix, prefixRecursive)
#         print(prefix, prefixRecursive)
        S = sum(counter.values())
        for word, frequency in counter.items():
            probsRecursive = recursiveProba(lines, lmda, n, delta)
            probs[prefix][word] = np.maximum(0, frequency - delta)/S + lmda * probsRecursive[prefixRecursive][word]
    return probs

In [263]:
out = recursiveProba(lines[0:1], lmda=1, n=2, delta = 0)
out

DEALING WITH N 2
Counter()
('_UNK_',)
('dual',)
('recurrent',)
('attention',)
('units',)
('for',)
('visual',)
('question',)
('answering',)
(';',)
('we',)
('propose',)
('an',)
('architecture',)
('vqa',)
('which',)
('utilizes',)
('layers',)
('to',)
('generate',)
('and',)
('textual',)
('.',)
('the',)
('memory',)
('characteristic',)
('of',)
('proposed',)
('offers',)
('a',)
('rich',)
('joint',)
('embedding',)
('features',)
('enables',)
('model',)
('reason',)
('relations',)
('between',)
('several',)
('parts',)
('image',)
('our',)
('single',)
('outperforms',)
('first',)
('place',)
('winner',)
('on',)
('1',)
('0',)
('dataset',)
(',',)
('performs',)
('within',)
('margin',)
('current',)
('state',)
('-',)
('art',)
('ensemble',)
('also',)
('experiment',)
('with',)
('replacing',)
('mechanisms',)
('in',)
('other',)
('models',)
('implementation',)
('show',)
('increased',)
('accuracy',)
('both',)
('cases',)
('mechanism',)
('improves',)
('performance',)
('tasks',)
('requiring',)
('sequential',)
('or',)

defaultdict(collections.Counter,
            {('_UNK_',): Counter({'dual': 1.0000844309354948}),
             ('dual',): Counter({'recurrent': 1.000337723741979}),
             ('recurrent',): Counter({'attention': 0.7504221546774739,
                      'layers': 0.25008443093549476}),
             ('attention',): Counter({'units': 0.40016886187098955,
                      '.': 0.2005065856129686,
                      'mechanisms': 0.20008443093549477,
                      'mechanism': 0.20008443093549477}),
             ('units',): Counter({'for': 0.5001688618709895,
                      'offers': 0.5000844309354948}),
             ('for',): Counter({'visual': 0.5002532928064843,
                      'vqa': 0.5002532928064843}),
             ('visual',): Counter({'question': 0.33350219520432284,
                      'and': 0.6670888213441405}),
             ('question',): Counter({'answering': 0.5000844309354948,
                      '.': 0.5005065856129686}),
             (

In [165]:
def recursiveProba(lines, lmda, n, delta):
    
    if n == 1:
        counts = count_ngrams(lines, n)
        probs = defaultdict(Counter)
        for prefix, counter in counts.items():
            S = sum(counter.values())
            for word, frequency in counter.items():
                probs[prefix][word] = frequency/S
        return probs

    n = n - 1
    counts = count_ngrams(lines, n)
    probs = defaultdict(Counter)
    vocab = set(token for token_counts in counts.values() for token in token_counts)
    for prefix, counter in counts.items():
        S = sum(counter.values())
        for word, frequency in counter.items():
            prefixRecursive = prefix[:n]
            probsRecursive = recursiveProba(lines, lmda, n, delta)
            probs[prefix][word] = np.maximum(0, frequency - delta)/S + (1/len(vocab)) * probsRecursive[prefixRecursive][word]
    return probs

class KneserNeyLanguageModel(NGramLanguageModel): 
    """ A template for Kneser-Ney language model. Default delta may be suboptimal. """
    def __init__(self, lines, n, delta=1.0):
        self.n = n
        counts = count_ngrams(lines, n)
        self.vocab = set(token for token_counts in counts.values() for token in token_counts)
        self.probs = recursiveProba(lines, 1, n, delta)
        
    def get_possible_next_tokens(self, prefix):
        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [ UNK ] * (self.n - 1 - len(prefix)) + prefix
        return self.probs[tuple(prefix)]
        
    def get_next_token_prob(self, prefix, next_token):
        return self.get_possible_next_tokens(prefix).get(next_token, 0)

In [ ]:
    def get_possible_next_tokens(self, prefix):
        token_probs = super().get_possible_next_tokens(prefix)
        missing_prob_total = 1.0 - sum(token_probs.values())
        missing_prob = missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        return {token: token_probs.get(token, missing_prob) for token in self.vocab}
    
    def get_next_token_prob(self, prefix, next_token):
        token_probs = super().get_possible_next_tokens(prefix)
        if next_token in token_probs:
            return token_probs[next_token]
        else:
            missing_prob_total = 1.0 - sum(token_probs.values())
            missing_prob_total = max(0, missing_prob_total) # prevent rounding errors
            return missing_prob_total / max(1, len(self.vocab) - len(token_probs))

In [167]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = KneserNeyLanguageModel(dummy_lines, n=n)
    print(n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

1
2


AssertionError: I told you not to break anything! :)

In [169]:
dummy_lm.vocab

{'"',
 '".',
 '$',
 '$)',
 '&',
 "'",
 "''",
 '(',
 '($',
 ')',
 ').',
 '*',
 ',',
 '-',
 '.',
 ".'",
 '.(',
 '0',
 '02141',
 '0225',
 '0711',
 '1',
 '1306',
 '1612',
 '17',
 '18',
 '1934',
 '2',
 '2001',
 '2011',
 '2016',
 '2017',
 '2058',
 '2061',
 '2270',
 '26th',
 '2d',
 '3',
 '38',
 '3d',
 '7',
 '94',
 ':',
 ';',
 '=',
 '=?',
 '?',
 '[',
 ']',
 '^',
 '_EOS_',
 '``',
 'a',
 'about',
 'abstract',
 'accommodation',
 'accuracy',
 'accurately',
 'achieve',
 'acl',
 'across',
 'act',
 'action',
 'actions',
 'activation',
 'activities',
 'activity',
 'activitynet',
 'adbiatic',
 'adjust',
 'adjusting',
 'administrators',
 'advances',
 'advantages',
 'adversarial',
 'after',
 'agent',
 'ai',
 'aim',
 'aims',
 'al',
 'algorithm',
 'algorithms',
 'ali',
 'all',
 'allocation',
 'almost',
 'alphabet',
 'alphabetic',
 'also',
 'alternatives',
 'always',
 'among',
 'an',
 'analysis',
 'analyze',
 'anchorage',
 'and',
 'angluin',
 'annotation',
 'answer',
 'any',
 'applications',
 'applied',
 'a

In [171]:
dummy_lm.get_next_token_prob('a', 'car')

0

In [172]:
for n in (1, 2, 3):
    lm = KneserNeyLanguageModel(train_lines, n=n, delta=1)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

/home/userlocal/miniconda3/envs/nlp/lib/python3.7/site-packages/ipykernel_launcher.py:18: RuntimeWarning: divide by zero encountered in log


N = 1, Perplexity = 1832.23136


KeyboardInterrupt: 